In [8]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import random
import numpy as np
from sympy import primerange
import itertools
from numba import jit, prange

In [9]:
cities = pd.read_csv("dataset/cities.csv")

In [23]:
tour = pd.read_csv("submission_181212.csv")['Path'].tolist()

In [11]:
primes = list(primerange(0, len(cities)))

In [12]:
cities['prime'] = cities.CityId.isin(primes).astype(int)

In [13]:
def score_tour(tour):
    # length of any given tour with primes calculation
    df = cities.reindex(tour + [0]).reset_index()
    df['prime'] = df.CityId.isin(primes).astype(int)
    df['dist'] = np.hypot(df.X - df.X.shift(-1), df.Y - df.Y.shift(-1))
    df['penalty'] = df['dist'][9::10] * (1 - df['prime'][9::10]) * 0.1
    return df.dist.sum() + df.penalty.sum()

In [26]:
@jit(nopython=True, parallel=True, fastmath=True)
def score_tour_numba(tour_, X, Y, primes):
    # length of any given tour with primes calculation
    full = 0.0
    for i in prange(0, len(tour_)-1):
        alpha = 1.0
        dst = np.hypot(X[tour_[i]] - X[tour_[i+1]], Y[tour_[i]] - Y[tour_[i+1]])
        if i%10 == 9 and primes[tour_[i]] == 0:
            alpha = 1.1
        full += alpha * dst
    return full

In [29]:
score_tour_numba(tour, cities.X.values, cities.Y.values, cities.prime.values)

1516697.7572615764

In [16]:
def local2op(path, i, k):
    tmp = path[i:k]
    return path[0:i] + tmp[::-1] + path[k:]

In [17]:
def possible_permutation(path, j, k):
    tmp_ = path[j:k].copy()
    per_tmp = [list(i) for i in itertools.permutations(tmp_)]
    for i in per_tmp:
        yield path[0:j] + i + path[k:]

In [18]:
for a in possible_permutation([1,2,3,4,5,6,7,8], 2,5):
    print(a)

[1, 2, 3, 4, 5, 6, 7, 8]
[1, 2, 3, 5, 4, 6, 7, 8]
[1, 2, 4, 3, 5, 6, 7, 8]
[1, 2, 4, 5, 3, 6, 7, 8]
[1, 2, 5, 3, 4, 6, 7, 8]
[1, 2, 5, 4, 3, 6, 7, 8]


In [19]:
# %timeit local2op(tour.copy(), 1, 5)

Permutation

In [ ]:
best_tour = score_tour_numba(tour, cities.X.values, cities.Y.values, cities.prime.values)
max_number = len(tour) - 1
tour_best = tour.copy()

for i in tqdm_notebook(range(18459,max_number), total=max_number - 18459 + 1):
    for k in range(i+1, i+5):
        if k - i > 1:
            for perm in possible_permutation(tour_best, i, k):
                fitness_tmp = score_tour_numba(perm, cities.X.values, cities.Y.values, cities.prime.values)
                if fitness_tmp < best_tour:
                    print(fitness_tmp)
                    best_tour = fitness_tmp
                    pd.DataFrame({"Path": perm}).to_csv("submition_181213_v_{}.csv".format(fitness_tmp), index=False)
                    tour_best = perm.copy()

In [ ]:
best_tour = score_tour(tour)
max_number = len(tour) - 1
tour_best = tour.copy()

for i in tqdm_notebook(range(1,max_number), total=max_number +1):
    for k in range(i+1, i+15):
        if k - i > 1:
            fliped_tour = local2op(tour_best.copy(), i, k)
            fitness_tmp = score_tour(fliped_tour)
            if fitness_tmp < best_tour:
                print(fitness_tmp)
                best_tour = fitness_tmp
                pd.DataFrame({"Path": fliped_tour}).to_csv("submition_181208_v_{}.csv".format(fitness_tmp), index=False)
                tour_best = fliped_tour.copy()

In [ ]:
44229 + 2656 + 30375 + 64789

In [ ]:
tour[1:10:-1]

In [ ]:
score_tour(tour_best[::-1])